In [ ]:
#slow

In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
from econml.orf import DMLOrthoForest
from econml.dml import CausalForestDML
from openbt.openbt import OPENBT
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from copy import deepcopy
import econml
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import GPy
import pickle
import scipy.linalg as la

ModuleNotFoundError: No module named 'GPy'

In [ ]:
def categorical2indicator(data, name, categorical_max=4):
    '''
    Transforms categorical variable with name 'name' form a data frame to indicator variables
    
    Taken from https://github.com/IirisSundin/active-learning-for-decision-making/blob/e0c83f58181f81da2f867da4c49f1333fa7d0ae6/src/util.py#L14
    '''
    values = data[name].values
    values[values>= categorical_max] = categorical_max
    uni = np.unique(values)
    for i, value in enumerate(uni):
        data[name+'.'+str(i)] = np.array((values==value), dtype=int)
    data.drop(name, axis=1)
    return data

def prepare_data(row_to_test, random_state,categorical_max=2):
    df = pd.read_csv("./data/ihdp_rc.csv")
    #names = ["treatment", "y_factual", "y_cfactual", "mu0", "mu1"] + [f'x{x}' for x in range(25)])
    df = categorical2indicator(df, 'birth.o', categorical_max=categorical_max)
    cnames = set(df.columns) - set(["y_factual", "y_cfactual", "treatment"])
    #X = df.loc[:,"x0":].to_numpy()
    df["first"] = df["first"] - 1
    
    # Creating training data
    X_test = df.loc[row_to_test, cnames]
    y_test = df.loc[row_to_test, "y_factual"]
    t_test = df.loc[row_to_test, "treatment"]
    ite_test = np.where(
                   df.loc[row_to_test, "treatment"] == 1,
                   df.loc[row_to_test, 'y_factual'] -  df.loc[row_to_test, "y_cfactual"],
                   df.loc[row_to_test, 'y_cfactual'] - df.loc[row_to_test, "y_factual"])
    
    df.drop(row_to_test, axis=0, inplace=True)
    X = df[cnames]
    t = df["treatment"].to_numpy()
    y = df["y_factual"].to_numpy()
    y1 = np.where(df["treatment"] == 1,
                   df['y_factual'],
                   df['y_cfactual'])
    y0 = np.where(df["treatment"] == 0,
                   df['y_factual'],
                   df['y_cfactual'])
    ite = np.where(df["treatment"] == 1,
                   df['y_factual'] - df["y_cfactual"],
                   df['y_cfactual'] - df["y_factual"])
    
    scalers = {}
    X_train, X_pool, y_train, y_pool, t_train, t_pool, \
     y1_train, y1_pool, y0_train, y0_pool, ite_train, ite_pool = train_test_split(X, y, t,
                                                                         y1, 
                                                                         y0,
                                                                         ite,
                                                                         test_size = 646,
                                                                         random_state=random_state)
    for col in ['bw','nnhealth', 'preterm', 'b.head','momage']:
        scalers[col] = StandardScaler()
        X_train[col] = scalers[col].fit_transform(X_train[col].to_numpy().reshape(-1,1))
        X_pool[col] = scalers[col].transform(X_pool[col].to_numpy().reshape(-1,1))
        try:
            X_test[col] = scalers[col].transform(X_test[col].to_numpy().reshape(-1,1))
        except:
            X_test[col] = scalers[col].transform(X_test[col].reshape(-1,1))
    #X_train, X_pool, y_train, y_pool = X[], X_pool, y_train, y_pool
    ds = {"X_training": X_train.to_numpy(),
     "y_training": y_train,
     "t_training": t_train,
     "ite_training": np.zeros_like(y_train),
     "X_pool": X_pool.to_numpy(), 
     "y_pool": y_pool,
     "t_pool": t_pool,
     "y1_pool": y1_pool,
     "y0_pool": y0_pool,
     "X_test": X_test.to_numpy().reshape(1, -1),
     "y_test": y_test,
     "t_test": t_test,
     "ite_test": ite_test.reshape(1, -1)
     }
    return ds

In [ ]:
class GPyEstimator(BaseITEEstimator):
    # https://github.com/IirisSundin/active-learning-for-decision-making/blob/master/src/gpmodel.py
    
    def _create_model(self, x, y):
        d = x.shape[1]
        prior = GPy.core.parameterization.priors.Gamma(a=1.5,b=3.0)
        kern = GPy.kern.RBF(input_dim=d, ARD=True)
        kern.variance.set_prior(prior, warning=False)
        kern.lengthscale.set_prior(prior, warning=False)
        lik1 = GPy.likelihoods.Gaussian()
        lik1.variance.set_prior(prior, warning=False)
        lik_expert = GPy.likelihoods.Gaussian()
        lik_expert.variance.set_prior(prior, warning=False)
        lik = GPy.likelihoods.MixedNoise([lik1, lik_expert])
        output_index = np.ones(x.shape[0])
        model = GPy.core.GP(X =  x,
                                 Y =  y.reshape(-1, 1), 
                                 kernel=kern,
                                 likelihood=lik, 
                                 Y_metadata = {'output_index':output_index})
        model.optimize()
        return model
        
    def fit(self, **kwargs):
        self.models = {}
        Xt, yt = kwargs["X_training"][(kwargs["t_training"] == 1), :],\
                kwargs["y_training"][(kwargs["t_training"] == 1)]
        Xc, yc = kwargs["X_training"][(kwargs["t_training"] == 0), :],\
        kwargs["y_training"][(kwargs["t_training"] == 0)]
        self.models["c"] = self._create_model(Xc, yc)
        self.models["t"] = self._create_model(Xt, yt)
        
    def predict(self, X, **kwargs):
        if len(X.shape) == 1:
            X = X.reshape((1, -1))
        p1 = self.models["t"].posterior_samples_f(X, 100)
        p0 = self.models["c"].posterior_samples_f(X, 100)
        ite = p1 - p0
        ite = ite.squeeze(1)
        if "return_mean" in kwargs:
            if kwargs["return_mean"]:
                ite = self.models["t"].predict_noiseless(X)[
                    0] - self.models["c"].predict_noiseless(X)[0]
        if "return_counterfactuals" in kwargs:
            if kwargs["return_counterfactuals"]:
                p1, p1s = self.models["t"]._raw_predict(X)
                p0, p0s = self.models["c"]._raw_predict(X)
                ite = (ite, p1, p0, p1s, p0s)
        return ite

In [ ]:
class ExpectedTargetedIG(BaseAcquisitionFunction):
    def mvn_KL(self, mu1, S1, mu2, S2):
        '''
        KL divergence between two multivariate normals
        '''
        if type(mu1)!=np.ndarray:
            d = 1
        else:
            d = mu1.shape[0]
        KL = 0.5*(np.log(la.det(S2)/la.det(S1)) + np.trace(la.inv(S2).dot(S1)) + (mu1 - mu2).T.dot(la.inv(S2).dot((mu1 - mu2))) - d)
        return KL
    
    def calculate_metrics(self, model, dataset):
        '''
        Uses Gauss-Hermite quadrature to compute expected information gain in decision
        '''
        ite_pred, p1, p0, p1s, p0s = model.predict(X = dataset["X_test"],
                                         return_counterfactuals=True)
        ypred = np.where(p1 > p0, p1, p0)
        Spred = np.where(p1 > p0, p1s, p0s)
        N = dataset["X_pool"].shape[0]
        # d = self._decide(preds)
        # ypred, Spred = preds[d]
        X_STAR = dataset["X_pool"]
        # x_star = self.dat['predictors'][n,:].reshape(1,-1)
        a_star = 1 - dataset["t_pool"] # counterfactual action

        points, weights = np.polynomial.hermite.hermgauss(32) 
        scores = []
        NUM_SIMULATIONS = N
        sims = np.random.choice(N, NUM_SIMULATIONS, replace=False)
        #for i in range(X_STAR.shape[0]):
        for i in sims:
            x_star = X_STAR[i, :].reshape((1, -1))
            targig = 0.0
            for ii, yy in enumerate(points):
                preds_star = model.predict(x_star, return_counterfactuals=True)
                if dataset["t_pool"][i] == 1:
                    mu_star, S_star = preds_star[1], preds_star[3]
                else:
                    mu_star, S_star = preds_star[2], preds_star[4]
                y_star = np.sqrt(2)*np.sqrt(S_star)*yy + mu_star #for substitution
                #try:
                new_model = deepcopy(model)
                new_data = {"X_training": np.concatenate((dataset["X_training"], x_star)),
                           "y_training":  np.concatenate((dataset["y_training"], 
                                                          dataset["y_pool"][i].ravel())),
                            "t_training": np.concatenate((dataset["t_training"], 
                                                          dataset["t_pool"][i].ravel()))
                           }
                new_model.fit(**new_data)
                preds_next = new_model.predict(X = dataset["X_test"],
                                 return_counterfactuals=True)
                #model_star = self.update(a_star, x_star, y_star)
                # preds_next = self.predict(new_predictors, model_star)
                #    preds_next = self.predict(new_predictors)
                D_KL = 0
                for d in range(2):
                    ypred_next, Spred_next = preds_next[d+1], preds_next[d+3]
                    D_KL += self.mvn_KL(ypred_next, Spred_next, ypred, Spred)
                targig += weights[ii] * 1/np.sqrt(np.pi) * D_KL
            scores += [targig]
        scs = np.zeros(N)
        scs[sims] = np.array(scores).ravel()
        return scs

In [ ]:
from xbart import XBART
class XBARTEstimator(BaseITEEstimator):
    def __init__(self, model, two_model=False):
        super().__init__(model = model, two_model = two_model,dataset=None, ps_model=None)
        
    def predict(self, **kwargs):
        X0 = np.concatenate((kwargs["X"],
                             np.zeros(kwargs["X"].shape[0]).reshape((-1,1))),axis=1)
        X1 = np.concatenate((kwargs["X"],
                             np.ones(kwargs["X"].shape[0]).reshape((-1,1))),axis=1)
        if "return_mean" in kwargs:
            out = self.model.predict(X1) - self.model.predict(X0)
        else:
            out = self.model.predict(X1, return_mean=False) - self.model.predict(X0, return_mean=False)
        return out
        
ests = {#"xbart":XBARTEstimator(model = XBART(num_sweeps=20), two_model = False),
        "gpy":GPyEstimator()}

In [ ]:
for key, value in ests.items():
    res = {}
    for i in range(5):
        for d in range(2):
            ds = prepare_data(d, i)
            asl = BaseActiveLearner(estimator = value,
                                    acquisition_function=[RandomAcquisitionFunction(name="random",
                                                                                    method = "top" ),
                                                          EMCMAcquisitionFunction(name="emcm",
                                                                                  method = "top"), 
                                                          ExpectedTargetedIG(name="etig", method="top")],
                                    assignment_function=BaseAssignmentFunction(),
                                    stopping_function = None,
                                    dataset=ds,
                                   al_steps=6)
            _ = asl.simulate(no_query=1, metric=["PEHE", "decision"])
            res[f"{i}_{d}"] =  pd.DataFrame(asl.simulation_results)
            res[f"{i}_{d}"]["sim"] = i
            res[f"{i}_{d}"]["data"] = d
            print(f"D {d} is done")

In [ ]:
pd.concat([x.loc["decision"].apply(pd.Series).T for x in res.values()])

,random_1,emcm_1,etig_1,sim,data
0,NaN,NaN,NaN,0.0,0.0
1,0.0,0.0,0.0,NaN,NaN
2,0.0,0.0,0.0,NaN,NaN
3,0.0,0.0,0.0,NaN,NaN
4,0.0,0.0,0.0,NaN,NaN
5,0.0,0.0,0.0,NaN,NaN
6,0.0,0.0,0.0,NaN,NaN
0,NaN,NaN,NaN,0.0,1.0
1,0.0,0.0,0.0,NaN,NaN
2,0.0,0.0,0.0,NaN,NaN


In [ ]:
pd.concat([x.loc["decision"].apply(pd.Series).T for x in res.values()]).reset_index().groupby('index').mean()

,random_1,unc_1,type_s_1,emcm_1,sim,data
index,,,,,,
0,NaN,NaN,NaN,NaN,0.0,17.5
1,0.750000,0.750000,0.750000,0.750000,NaN,NaN
2,0.527778,0.750000,0.777778,0.750000,NaN,NaN
3,0.472222,0.611111,0.750000,0.388889,NaN,NaN
4,0.722222,0.583333,0.472222,0.555556,NaN,NaN
5,0.555556,0.555556,0.722222,0.416667,NaN,NaN
6,0.305556,0.333333,0.416667,0.527778,NaN,NaN
